In [1]:
import graphlab as gl
sales=gl.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to vpsingh12157@gmail.com and will expire on June 28, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\HP\AppData\Local\Temp\graphlab_server_1533726208.log.0


In [2]:
sales['floors']=sales['floors'].astype(float)

In [3]:
import numpy as np

In [4]:
def get_numpy_data(data_frame ,input_feature ,output ):
    data_frame['constant']=1
    input_feature=['constant']+input_feature
    data_frame_sel_feat=data_frame.select_columns(input_feature)
    data_frame_sel_feat_matrix=data_frame_sel_feat.to_numpy()
    output_numarray=data_frame[output].to_numpy()
    
    return (data_frame_sel_feat_matrix ,output_numarray)

In [5]:
def predict_output(feature_matrix ,weights):
    return(np.dot(feature_matrix ,weights ))

In [6]:
x=np.array(([1,2,6],[8,5,9]))

In [7]:
norm2=np.linalg.norm(x,axis=0)
print norm2

[ 8.06225775  5.38516481 10.81665383]


In [9]:
print x/norm2

[[0.12403473 0.37139068 0.5547002 ]
 [0.99227788 0.92847669 0.83205029]]


In [10]:
def normalized_features(feature_matrix):
    norm2=np.linalg.norm(feature_matrix ,axis=0)
    feature_matrix=feature_matrix/norm2
    return(feature_matrix ,norm2)

In [11]:
test_feature_matrix,test_norm=normalized_features(np.array(([3.,6.,9.],[4.,8.,12.])))

In [12]:
print test_feature_matrix

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]


In [13]:
print test_norm

[ 5. 10. 15.]


In [14]:
simple_features=['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output_array) = get_numpy_data(sales, simple_features, my_output)

In [15]:
(normalized_simple_feature_matrix ,norm)=normalized_features(simple_feature_matrix)

In [16]:
weights = np.array([1., 4., 1.])


In [17]:
prediction=predict_output(normalized_simple_feature_matrix ,weights )
roi=[]

In [18]:
for i in range(3):
    roi.append(np.dot(normalized_simple_feature_matrix[:,i],(output_array-prediction+np.dot(weights[i],normalized_simple_feature_matrix[:,i]))))

In [19]:
print roi

[79400300.03492919, 87939470.77299106, 80966698.67596567]


In [176]:
def lasso_cordinate_descent_step(i ,feature_matrix ,output_array ,weights ,l1_penalty):
    prediction=predict_output(feature_matrix ,weights)
    ro_i=np.dot(feature_matrix[:,i] ,(output_array-prediction+np.dot(feature_matrix[:,i],weights[i])))
    if i==0:
        new_weight=ro_i
        print "added ro"+str(ro_i)+"in "+str(i)+"th iteration" 
    elif ro_i > l1_penalty/2.:
        new_weight=ro_i-(l1_penalty/2.)
        print "subtracted "+str(l1_penalty/2.)+"in "+str(i)+"th iteration" 
    elif ro_i < -l1_penalty/2.:
        new_weight=ro_i+(l1_penalty/2.)
        print "added "+str(l1_penalty/2.)+"in "+str(i)+"th iteration" 
    else:
        new_weight=0
        print "weight =0"+"in "+str(i)+"th iteration" 
        
    return new_weight
        

In [177]:
import math
print lasso_cordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

subtracted 0.05in 1th iteration
0.4255588466910251


In [200]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, weights, l1_penalty, tolerance):
    max_change=tolerance+1
    while(max_change>tolerance):
        max_change=-100000000
        for i in range(len(weights)):
            old_weight=weights[i]
            weights[i]=lasso_cordinate_descent_step(i,feature_matrix,output ,weights ,l1_penalty) 
            change_in_weight=abs(old_weight-weights[i])
            if change_in_weight>max_change:
                max_change=change_in_weight
        print max_change
    return weights
        

In [188]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [189]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalized_features(simple_feature_matrix)

In [201]:
weights=lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix ,output ,initial_weights ,l1_penalty,tolerance)

added ro21624997.647071432in 0th iteration
subtracted 5000000.0in 1th iteration
weight =0in 2th iteration
0.7192914895713329


In [243]:
prediction=predict_output(normalized_simple_feature_matrix ,weights)
rss=prediction-output
rss=rss**2
rss.sum()

1630492474904515.8

In [192]:
train_data,test_data = sales.random_split(.8,seed=0)

In [193]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [194]:
(final_feature_matrix, output_array) = get_numpy_data(train_data, all_features, my_output)

In [195]:
normalized_feature_matrix ,norm2 =normalized_features(final_feature_matrix)
initial_weight=np.zeros(len(all_features)+1)

In [216]:
weightle7=lasso_cyclical_coordinate_descent(normalized_feature_matrix ,output_array ,initial_weight,1e7,1.)

added ro78647853.7931799in 0th iteration
added 5000000.0in 1th iteration
subtracted 5000000.0in 2th iteration
subtracted 5000000.0in 3th iteration
weight =0in 4th iteration
weight =0in 5th iteration
subtracted 5000000.0in 6th iteration
subtracted 5000000.0in 7th iteration
subtracted 5000000.0in 8th iteration
subtracted 5000000.0in 9th iteration
added 5000000.0in 10th iteration
added 5000000.0in 11th iteration
added 5000000.0in 12th iteration
weight =0in 13th iteration
9883014.78170229
added ro74327065.00775187in 0th iteration
added 5000000.0in 1th iteration
weight =0in 2th iteration
subtracted 5000000.0in 3th iteration
weight =0in 4th iteration
subtracted 5000000.0in 5th iteration
subtracted 5000000.0in 6th iteration
subtracted 5000000.0in 7th iteration
subtracted 5000000.0in 8th iteration
subtracted 5000000.0in 9th iteration
weight =0in 10th iteration
weight =0in 11th iteration
added 5000000.0in 12th iteration
weight =0in 13th iteration
7054356.655243397
added ro69437108.91112638in 0t

weight =0in 9th iteration
subtracted 5000000.0in 10th iteration
subtracted 5000000.0in 11th iteration
weight =0in 12th iteration
weight =0in 13th iteration
485067.61780466884
added ro26553190.572341517in 0th iteration
weight =0in 1th iteration
weight =0in 2th iteration
subtracted 5000000.0in 3th iteration
weight =0in 4th iteration
weight =0in 5th iteration
subtracted 5000000.0in 6th iteration
subtracted 5000000.0in 7th iteration
weight =0in 8th iteration
weight =0in 9th iteration
subtracted 5000000.0in 10th iteration
subtracted 5000000.0in 11th iteration
weight =0in 12th iteration
weight =0in 13th iteration
484000.8229621155
added ro26700276.49032183in 0th iteration
weight =0in 1th iteration
weight =0in 2th iteration
subtracted 5000000.0in 3th iteration
weight =0in 4th iteration
weight =0in 5th iteration
subtracted 5000000.0in 6th iteration
subtracted 5000000.0in 7th iteration
weight =0in 8th iteration
weight =0in 9th iteration
subtracted 5000000.0in 10th iteration
subtracted 5000000.0

added ro24479465.815731257in 0th iteration
weight =0in 1th iteration
weight =0in 2th iteration
subtracted 5000000.0in 3th iteration
weight =0in 4th iteration
weight =0in 5th iteration
subtracted 5000000.0in 6th iteration
subtracted 5000000.0in 7th iteration
weight =0in 8th iteration
weight =0in 9th iteration
weight =0in 10th iteration
weight =0in 11th iteration
weight =0in 12th iteration
weight =0in 13th iteration
9291.860177658498
added ro24471633.324453555in 0th iteration
weight =0in 1th iteration
weight =0in 2th iteration
subtracted 5000000.0in 3th iteration
weight =0in 4th iteration
weight =0in 5th iteration
subtracted 5000000.0in 6th iteration
subtracted 5000000.0in 7th iteration
weight =0in 8th iteration
weight =0in 9th iteration
weight =0in 10th iteration
weight =0in 11th iteration
weight =0in 12th iteration
weight =0in 13th iteration
7832.491277702153
added ro24465030.995270565in 0th iteration
weight =0in 1th iteration
weight =0in 2th iteration
subtracted 5000000.0in 3th iterat

subtracted 5000000.0in 7th iteration
weight =0in 8th iteration
weight =0in 9th iteration
weight =0in 10th iteration
weight =0in 11th iteration
weight =0in 12th iteration
weight =0in 13th iteration
33.06831347197294
added ro24429745.634695336in 0th iteration
weight =0in 1th iteration
weight =0in 2th iteration
subtracted 5000000.0in 3th iteration
weight =0in 4th iteration
weight =0in 5th iteration
subtracted 5000000.0in 6th iteration
subtracted 5000000.0in 7th iteration
weight =0in 8th iteration
weight =0in 9th iteration
weight =0in 10th iteration
weight =0in 11th iteration
weight =0in 12th iteration
weight =0in 13th iteration
27.874642092734575
added ro24429722.138012562in 0th iteration
weight =0in 1th iteration
weight =0in 2th iteration
subtracted 5000000.0in 3th iteration
weight =0in 4th iteration
weight =0in 5th iteration
subtracted 5000000.0in 6th iteration
subtracted 5000000.0in 7th iteration
weight =0in 8th iteration
weight =0in 9th iteration
weight =0in 10th iteration
weight =0in

In [217]:
weightle7

array([24429600.93820254,        0.        ,        0.        ,
       48389174.04258875,        0.        ,        0.        ,
        3317511.162289  ,  7329962.0051915 ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [227]:
weightle8=lasso_cyclical_coordinate_descent(normalized_feature_matrix ,output_array ,initial_weight,1e8,1.)

added ro24429600.167384926in 0th iteration
weight =0in 1th iteration
weight =0in 2th iteration
subtracted 50000000.0in 3th iteration
weight =0in 4th iteration
weight =0in 5th iteration
weight =0in 6th iteration
weight =0in 7th iteration
weight =0in 8th iteration
weight =0in 9th iteration
subtracted 50000000.0in 10th iteration
weight =0in 11th iteration
weight =0in 12th iteration
weight =0in 13th iteration
44999999.27413344
added ro66708837.38770002in 0th iteration
weight =0in 1th iteration
weight =0in 2th iteration
weight =0in 3th iteration
weight =0in 4th iteration
weight =0in 5th iteration
weight =0in 6th iteration
weight =0in 7th iteration
weight =0in 8th iteration
weight =0in 9th iteration
weight =0in 10th iteration
weight =0in 11th iteration
weight =0in 12th iteration
weight =0in 13th iteration
42279237.2203151
added ro71114625.75280938in 0th iteration
weight =0in 1th iteration
weight =0in 2th iteration
weight =0in 3th iteration
weight =0in 4th iteration
weight =0in 5th iteration


In [228]:
weightle8

array([71114625.75280938,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [231]:
weight1e4=lasso_cyclical_coordinate_descent(normalized_feature_matrix ,output_array ,initial_weight,1e4,5e5)

added ro71114625.75280938in 0th iteration
subtracted 5000.0in 1th iteration
subtracted 5000.0in 2th iteration
subtracted 5000.0in 3th iteration
added 5000.0in 4th iteration
added 5000.0in 5th iteration
subtracted 5000.0in 6th iteration
subtracted 5000.0in 7th iteration
added 5000.0in 8th iteration
subtracted 5000.0in 9th iteration
subtracted 5000.0in 10th iteration
subtracted 5000.0in 11th iteration
added 5000.0in 12th iteration
subtracted 5000.0in 13th iteration
12537789.747781776
added ro70455350.90750061in 0th iteration
subtracted 5000.0in 1th iteration
subtracted 5000.0in 2th iteration
subtracted 5000.0in 3th iteration
added 5000.0in 4th iteration
added 5000.0in 5th iteration
subtracted 5000.0in 6th iteration
subtracted 5000.0in 7th iteration
added 5000.0in 8th iteration
subtracted 5000.0in 9th iteration
subtracted 5000.0in 10th iteration
subtracted 5000.0in 11th iteration
added 5000.0in 12th iteration
subtracted 5000.0in 13th iteration
8441276.005488854
added ro70403779.35221073in

added ro73130751.77740999in 0th iteration
added 5000.0in 1th iteration
subtracted 5000.0in 2th iteration
subtracted 5000.0in 3th iteration
added 5000.0in 4th iteration
added 5000.0in 5th iteration
subtracted 5000.0in 6th iteration
subtracted 5000.0in 7th iteration
added 5000.0in 8th iteration
subtracted 5000.0in 9th iteration
subtracted 5000.0in 10th iteration
subtracted 5000.0in 11th iteration
added 5000.0in 12th iteration
subtracted 5000.0in 13th iteration
1081690.2924136221
added ro73238000.65325844in 0th iteration
added 5000.0in 1th iteration
subtracted 5000.0in 2th iteration
subtracted 5000.0in 3th iteration
added 5000.0in 4th iteration
added 5000.0in 5th iteration
subtracted 5000.0in 6th iteration
subtracted 5000.0in 7th iteration
added 5000.0in 8th iteration
subtracted 5000.0in 9th iteration
subtracted 5000.0in 10th iteration
subtracted 5000.0in 11th iteration
added 5000.0in 12th iteration
subtracted 5000.0in 13th iteration
1070005.5468560606
added ro73344534.02005516in 0th iter

subtracted 5000.0in 8th iteration
subtracted 5000.0in 9th iteration
added 5000.0in 10th iteration
added 5000.0in 11th iteration
added 5000.0in 12th iteration
subtracted 5000.0in 13th iteration
686510.6998799741
added ro76335912.63121067in 0th iteration
added 5000.0in 1th iteration
subtracted 5000.0in 2th iteration
subtracted 5000.0in 3th iteration
added 5000.0in 4th iteration
added 5000.0in 5th iteration
subtracted 5000.0in 6th iteration
subtracted 5000.0in 7th iteration
subtracted 5000.0in 8th iteration
subtracted 5000.0in 9th iteration
added 5000.0in 10th iteration
added 5000.0in 11th iteration
added 5000.0in 12th iteration
subtracted 5000.0in 13th iteration
676751.2429029197
added ro76424855.94485943in 0th iteration
added 5000.0in 1th iteration
subtracted 5000.0in 2th iteration
subtracted 5000.0in 3th iteration
added 5000.0in 4th iteration
added 5000.0in 5th iteration
subtracted 5000.0in 6th iteration
subtracted 5000.0in 7th iteration
subtracted 5000.0in 8th iteration
subtracted 500

In [232]:
weight1e4

array([ 78564738.47260302, -22097399.1236062 ,  12791071.78258317,
        93808087.79432714,  -2013172.81209796,  -4219184.78179709,
         6482842.76876443,   7127408.6920586 ,   5001665.5275527 ,
        14327517.39644272, -15770958.97405391,  -5159591.03562758,
       -84495341.3553755 ,   2824439.4077956 ])

In [219]:
norm2

array([1.31848398e+02, 4.60040216e+02, 2.96850552e+02, 2.99962419e+05,
       5.81709718e+06, 2.09458827e+02, 1.15325626e+01, 1.05933942e+02,
       4.57793622e+02, 1.02101959e+03, 2.59726472e+05, 7.01224951e+04,
       2.59922094e+05, 5.36953839e+04])

In [218]:
weightle7

array([24429600.93820254,        0.        ,        0.        ,
       48389174.04258875,        0.        ,        0.        ,
        3317511.162289  ,  7329962.0051915 ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [221]:
normalized_weight1e7=weightle7/norm2

In [225]:
normalized_weight1e7

array([1.85285535e+05, 0.00000000e+00, 0.00000000e+00, 1.61317455e+02,
       0.00000000e+00, 0.00000000e+00, 2.87664700e+05, 6.91937059e+04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00])

In [229]:
weightle8

array([71114625.75280938,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [230]:
normalized_weight1e8=weightle8/norm2
normalized_weight1e8

array([539366.62822135,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ])

In [233]:
weight1e4

array([ 78564738.47260302, -22097399.1236062 ,  12791071.78258317,
        93808087.79432714,  -2013172.81209796,  -4219184.78179709,
         6482842.76876443,   7127408.6920586 ,   5001665.5275527 ,
        14327517.39644272, -15770958.97405391,  -5159591.03562758,
       -84495341.3553755 ,   2824439.4077956 ])

In [234]:
normalized_weight1e4=weight1e4/norm2
normalized_weight1e4

array([ 5.95871772e+05, -4.80336248e+04,  4.30892640e+04,  3.12732802e+02,
       -3.46078594e-01, -2.01432656e+04,  5.62133760e+05,  6.72816340e+04,
        1.09255902e+04,  1.40325588e+04, -6.07214152e+01, -7.35796840e+01,
       -3.25079489e+02,  5.26011587e+01])

In [235]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

In [238]:
prediction1=predict_output(test_feature_matrix ,normalized_weight1e7)
rss=prediction1-test_output
rss=rss**2
rss.sum()

275962080509410.7

In [239]:
prediction2=predict_output(test_feature_matrix ,normalized_weight1e8)
rss=prediction2-test_output
rss=rss**2
rss.sum()

537166150034084.9

In [240]:
prediction3=predict_output(test_feature_matrix ,normalized_weight1e4)
rss=prediction3-test_output
rss=rss**2
rss.sum()

228459962944943.2